In [1]:
import scipy as stats
import pandas as pd
from pandas_datareader import data as pdr
import seaborn as sns; sns.set()
import matplotlib
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
import time
from datetime import date, datetime
from openpyxl import Workbook, load_workbook
yf.pdr_override()

In [2]:
def cambio():
    #euro/dólar BRL=X
    pdr.get_data_yahoo('EURUSD=X', start='2022-01-27', end= date.today())
    pdr.get_data_yahoo('BRL=X', start='2022-01-27', end= date.today())
    
    
def commodities():
    #SOJA

    # ZS=F  << ticker

    pdr.get_data_yahoo('ZS=F', start='2022-01-27')
    #MILHO

    # ZC=F  << ticker

    pdr.get_data_yahoo('ZC=F', start='2022-01-27')
    #CAFÉ
     
    #  KC=F << ticker

    pdr.get_data_yahoo('KC=F', start='2022-01-27')



In [3]:
def consulta_bc(codigo_bcb):
  url = 'http://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json'.format(codigo_bcb)
  df = pd.read_json(url)
  df['data'] = pd.to_datetime(df['data'], dayfirst=True)
  df.set_index('data')

  selic_meta = consulta_bc(432)
  teste = set(list(selic_meta['valor']))
    ###
  ipca = consulta_bc(433)
  ipca= ipca.loc[ipca['data'] > '2002-01-01']
  ipca.plot(x = 'data', y='valor', figsize= (18,5))

  return df

In [4]:
class Rentabilidade:
    
    def __init__(self, start, end):
        self.start = start
        self.end = end
        
# função para pegar a rentabilidade das ações desde o primeiro dia de investimento  

    def calc_rent_acao(self, acao_1,acao_2,acao_3,acao_4):
        
        cota = 100000000*0.3  ## valor em real
        
        acoes = [acao_1, acao_2, acao_3, acao_4]
        acao_open = pd.DataFrame()
        acao_close = pd.DataFrame()
        
        
        pesos = [0.25,0.25,0.25,0.25]
        
        
        for i in acoes:
            acao_open[i] = pdr.get_data_yahoo(i, start= self.start, end= self.end)['Open']
            acao_close[i] = pdr.get_data_yahoo(i, start= self.start, end= self.end)['Adj Close']
            
            
        rentabilidade = ((acao_close.iloc[-1]/acao_open.iloc[0]) - 1)
        
        retorno = round(np.dot(rentabilidade, pesos),2)
        
        return retorno
        
# função para pegar a rentabilidade das commodities desde o primeiro dia de investimento.

    def calc_rent_commodities(self, cmdts_1, cmdts_2, cmdts_3):
        
        cota = 100000000*0.2 ## valor em real

# pegando abertura e fechamento das commodities negociadas em dólar.

        commodities = [cmdts_1, cmdts_2, cmdts_3]
        cmdts_open = pd.DataFrame()
        cmdts_close = pd.DataFrame()

        for i in commodities:
            
            cmdts_open[i] = pdr.get_data_yahoo(i, start= self.start, end= self.end)['Open']
            cmdts_close[i] = pdr.get_data_yahoo(i, start= self.start, end= self.end)['Adj Close']
            
            
# pegando a abertura e fechamento do par USD/BRL > dólar/real para calcular o valor da cota em dólar.
        cambio = ['BRL=X']
    
        cambio_open = pd.DataFrame()
        cambio_close = pd.DataFrame()

        for i in cambio:
            
            cambio_open[i] = pdr.get_data_yahoo(i, start= self.start, end= self.end)['Open']
            cambio_close[i] = pdr.get_data_yahoo(i, start= self.start, end= self.end)['Adj Close']
        
#calculando a rentabilidade das commodities        
         
        rent_cambio = float(cambio_close.iloc[-1]/cambio_open.iloc[0])     
        rent_cmdts = cmdts_close.iloc[-1]/cmdts_open.iloc[0] 
        
        #rentabilidade referente a variação tanto das commodities quanto do dólar. Obs: a rent_cambio precisa ser
        #um valor float, pois a multiplicação é de um escalar por uma matriz, sendo necessário multiplicar cada coluna pelo escalar
        
        rentabilidade = rent_cambio*rent_cmdts - 1
        
        pesos = np.array([0.25, 0.25, 0.25])
        retorno = np.dot(rentabilidade, pesos)
          
        
       
        
        return round(retorno,4)
    
    def calc_rent_cambio(self, camb):
        
        ### cambio = ['BRL=X'] << USD/BRL
        
        cambio = [camb]
        
        cambio_open = pd.DataFrame()
        cambio_close = pd.DataFrame()

        for i in cambio:
            
            cambio_open[i] = pdr.get_data_yahoo(i, start= self.start, end= self.end)['Open']
            cambio_close[i] = pdr.get_data_yahoo(i, start= self.start, end= self.end)['Adj Close']
        
        rent = (cambio_close.iloc[-1]/cambio_open.iloc[0])-1
        
        return rent
        
    def consulta_bc(self, codigo_bcb):
        url = 'http://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json'.format(codigo_bcb)
        df = pd.read_json(url)
        df['data'] = pd.to_datetime(df['data'], dayfirst=True)
        df = df.loc[df['data'] >= self.start]
        
        c = 5000000
        i= float(df.iloc[-1,1]/100)/252
        
        # Data inicial
        d2 = datetime.strptime(self.start, '%Y-%m-%d')
        # Data final
        d1 = datetime.strptime(self.end, '%Y-%m-%d')
        t = abs((d2 - d1).days)
    
        m = c*((1+i)**t)
        ret = m/c -1
        return round(ret,4)


    def calc_rent_etf(self, etf_1):
    
        etf = [etf_1]
        
# pegando abertura e fechamento dos etfs negociadas em dólar.

        etf_open = pd.DataFrame()
        etf_close = pd.DataFrame()

        for i in etf:
            
            etf_open[i] = pdr.get_data_yahoo(i, start= self.start, end= self.end)['Open']
            etf_close[i] = pdr.get_data_yahoo(i, start= self.start, end= self.end)['Adj Close']
            
            
# pegando a abertura e fechamento do par USD/BRL > dólar/real para calcular o valor da cota em dólar.
        cambio = ['BRL=X']
    
        cambio_open = pd.DataFrame()
        cambio_close = pd.DataFrame()

        for i in cambio:
            
            cambio_open[i] = pdr.get_data_yahoo(i, start= self.start, end= self.end)['Open']
            cambio_close[i] = pdr.get_data_yahoo(i, start= self.start, end= self.end)['Adj Close']
        
#calculando a rentabilidade dos etf       
         
        rent_cambio = float(cambio_close.iloc[-1]/cambio_open.iloc[0])     
        rent_etf = etf_close.iloc[-1]/etf_open.iloc[0] 
        
        #rentabilidade referente a variação tanto das commodities quanto do dólar. Obs: a rent_cambio precisa ser
        #um valor float, pois a multiplicação é de um escalar por uma matriz, sendo necessário multiplicar cada coluna pelo escalar
        
        rentabilidade = rent_cambio*rent_etf - 1
        
        return rentabilidade

In [6]:
class Rentabilidade_1:
    
    def __init__(self, start, end):
        self.start = start
        self.end = end
        
# função para pegar a rentabilidade das ações desde o primeiro dia de investimento  

    def calc_rent_acao(self, acao_1,acao_2,acao_3,acao_4):
        
        cota = 100000000*0.3  ## valor em real
        
        acoes = [acao_1, acao_2, acao_3, acao_4]
        acao_open = pd.DataFrame()
        acao_close = pd.DataFrame()
        
        
        pesos = [0.25,0.25,0.10,0.25]
        
        
        for i in acoes:
            acao_open[i] = pdr.get_data_yahoo(i, start= self.start, end= self.end)['Open']
            acao_close[i] = pdr.get_data_yahoo(i, start= self.start, end= self.end)['Adj Close']
            
            
        rentabilidade = ((acao_close.iloc[-1]/acao_open.iloc[0]) - 1)
        
        retorno = round(np.dot(rentabilidade, pesos),2)
        
        return retorno
        
# função para pegar a rentabilidade das commodities desde o primeiro dia de investimento.

    def calc_rent_commodities(self, cmdts_1, cmdts_2, cmdts_3):
        
        cota = 100000000*0.2 ## valor em real

# pegando abertura e fechamento das commodities negociadas em dólar.

        commodities = [cmdts_1, cmdts_2, cmdts_3]
        cmdts_open = pd.DataFrame()
        cmdts_close = pd.DataFrame()

        for i in commodities:
            
            cmdts_open[i] = pdr.get_data_yahoo(i, start= self.start, end= self.end)['Open']
            cmdts_close[i] = pdr.get_data_yahoo(i, start= self.start, end= self.end)['Adj Close']
            
            
# pegando a abertura e fechamento do par USD/BRL > dólar/real para calcular o valor da cota em dólar.
        cambio = ['BRL=X']
    
        cambio_open = pd.DataFrame()
        cambio_close = pd.DataFrame()

        for i in cambio:
            
            cambio_open[i] = pdr.get_data_yahoo(i, start= self.start, end= self.end)['Open']
            cambio_close[i] = pdr.get_data_yahoo(i, start= self.start, end= self.end)['Adj Close']
        
#calculando a rentabilidade das commodities        
         
        rent_cambio = float(cambio_close.iloc[-1]/cambio_open.iloc[0])     
        rent_cmdts = cmdts_close.iloc[-1]/cmdts_open.iloc[0] 
        
        #rentabilidade referente a variação tanto das commodities quanto do dólar. Obs: a rent_cambio precisa ser
        #um valor float, pois a multiplicação é de um escalar por uma matriz, sendo necessário multiplicar cada coluna pelo escalar
        
        rentabilidade = rent_cambio*rent_cmdts - 1
        
        pesos = np.array([0.25, 0.25, 0.25])
        retorno = np.dot(rentabilidade, pesos)
          
        
       
        
        return round(retorno,4)
    
    def calc_rent_cambio(self, camb):
        
        ### cambio = ['BRL=X'] << USD/BRL
        
        cambio = [camb]
        
        cambio_open = pd.DataFrame()
        cambio_close = pd.DataFrame()

        for i in cambio:
            
            cambio_open[i] = pdr.get_data_yahoo(i, start= self.start, end= self.end)['Open']
            cambio_close[i] = pdr.get_data_yahoo(i, start= self.start, end= self.end)['Adj Close']
        
        rent = (cambio_close.iloc[-1]/cambio_open.iloc[0])-1
        
        return rent
        
    def consulta_bc(self, codigo_bcb):
        url = 'http://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json'.format(codigo_bcb)
        df = pd.read_json(url)
        df['data'] = pd.to_datetime(df['data'], dayfirst=True)
        df = df.loc[df['data'] >= self.start]
        
        c = 5000000
        i= float(df.iloc[-1,1]/100)/252
        
        # Data inicial
        d2 = datetime.strptime(self.start, '%Y-%m-%d')
        # Data final
        d1 = datetime.strptime(self.end, '%Y-%m-%d')
        t = abs((d2 - d1).days)
    
        m = c*((1+i)**t)
        ret = m/c -1
        return round(ret,4)


    def calc_rent_etf(self, etf_1):
    
        etf = [etf_1]
        
# pegando abertura e fechamento dos etfs negociadas em dólar.

        etf_open = pd.DataFrame()
        etf_close = pd.DataFrame()

        for i in etf:
            
            etf_open[i] = pdr.get_data_yahoo(i, start= self.start, end= self.end)['Open']
            etf_close[i] = pdr.get_data_yahoo(i, start= self.start, end= self.end)['Adj Close']
            
            
# pegando a abertura e fechamento do par USD/BRL > dólar/real para calcular o valor da cota em dólar.
        cambio = ['BRL=X']
    
        cambio_open = pd.DataFrame()
        cambio_close = pd.DataFrame()

        for i in cambio:
            
            cambio_open[i] = pdr.get_data_yahoo(i, start= self.start, end= self.end)['Open']
            cambio_close[i] = pdr.get_data_yahoo(i, start= self.start, end= self.end)['Adj Close']
        
#calculando a rentabilidade dos etf       
         
        rent_cambio = float(cambio_close.iloc[-1]/cambio_open.iloc[0])     
        rent_etf = etf_close.iloc[-1]/etf_open.iloc[0] 
        
        #rentabilidade referente a variação tanto das commodities quanto do dólar. Obs: a rent_cambio precisa ser
        #um valor float, pois a multiplicação é de um escalar por uma matriz, sendo necessário multiplicar cada coluna pelo escalar
        pesos = [0.5]
        rentabilidade = rent_cambio*rent_etf - 1
        rent = rentabilidade*0.5
        return rent